# lab 6 - wide and deep

## Overview

This data set was created by IBM data scientists.  It describes 35 features for 1470 (fictional) employees including whether or not the employee has left the firm (labeled "attrition" in the dataset).  Employees leave companies for a variety of reasons: disatisfaction with their role, their manager or their pay.  Perhaps they aren't necessarily dissatified with their current job but feel like something better is out there.  Or maybe they just feel like they'd been there long enough, and want something different. Most likely its a combination of all of these things, plus a few others.  

Employers would like to have a sense of why and when an employee might leave.  If an employer believes that an employee that they really value might leave, they could respond and try to prevent them from leaving.  This is what we will attempt to predict using a wide and deep neural network.

In [87]:
import pandas as pd
import numpy as np
import os

In [88]:
data_path = '../data/'
df = pd.read_csv(os.path.join(data_path, 'WA_Fn-UseC_-HR-Employee-Attrition.csv'))
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [89]:
print('Breakdown of attrition.  (0) Stayed at company, (1) Left company')

df.Attrition.value_counts()

Breakdown of attrition.  (0) Stayed at company, (1) Left company


No     1233
Yes     237
Name: Attrition, dtype: int64

# Expansion

This is a relative small dataset with only 1470 data rows. We want more data to train and test. For data expension, we will try several ways to do so. And first we are going to copy and append some rows to origin dataset and expand it to 2000 rows.
For another attempt, we take numberical datas from the raw dataframe slice, and add some randomly generat noise to these numerical data. Then we insert categorical data rows back to the dataframe of numerical data with random noise.

In [90]:
df_slice = df[:530]

In [91]:
df_new = df.append(df_slice)
df_new = df_new.reset_index(drop=True)

In [92]:
df_slice2 = df_slice[['Age','DistanceFromHome','Education','EnvironmentSatisfaction',
                      'JobSatisfaction','MonthlyIncome','PerformanceRating','RelationshipSatisfaction',
                      'TotalWorkingYears','YearsAtCompany']]

We use df_slice2 to take numberical datas from the raw dataframe slice, and add some randomly generated noise to these numerical data.

In [93]:
df_slice2 = df_slice2 * (1 + np.random.uniform(-0.01,0.01,(df_slice2.shape)))

In [94]:
df_slice2.insert(1, 'Attrition', df_slice['Attrition'])
df_slice2.insert(2, 'Department', df_slice['Department'])
df_slice2.insert(6, 'Gender', df_slice['Gender'])
df_slice2.insert(8, 'MaritalStatus', df_slice['MaritalStatus'])
df_slice2.insert(10, 'OverTime', df_slice['OverTime'])

In [95]:
df_new2 = df.append(df_slice2)
df_new2 = df_new2.reset_index(drop=True)
df = df_new2

So now we have a dataframe with 2000 rows and 15 clomuns expanded dataset

There are 35 features in total in the dataset, but we don't want to use all of them.  
Let's focus on a few of them:
- Age
- Attrition 
- Department
- DistanceFromHome 
- Education 
- EduacationField
- EnvironmentSatisfaction
- Gender
- JobSatisfaction
- MaritalStatus
- MonthlyIncome
- OverTime
- PerformanceRating
- RelationshipSatisfaction
- TotalWorkingYears
- YearsAtCompany
- YearsSinceLastPromotion

These features are what we believe important to predict the attrition status. 
We will use attrition as our label.

So let's first drop the other features. 

In [96]:
to_keep = {'Age', 'Attrition', 'Department','DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'Gender', 'JobSatisfaction', 'MaritalStatus',
           'MonthlyIncome', 'OverTime', 'PerformanceRating', 'RelationshipSatisfaction','TotalWorkingYears','YearsAtCompany'}
to_drop = set(df.columns)-to_keep
df.drop(to_drop, axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 15 columns):
Age                         2000 non-null float64
Attrition                   2000 non-null object
Department                  2000 non-null object
DistanceFromHome            2000 non-null float64
Education                   2000 non-null float64
EnvironmentSatisfaction     2000 non-null float64
Gender                      2000 non-null object
JobSatisfaction             2000 non-null float64
MaritalStatus               2000 non-null object
MonthlyIncome               2000 non-null float64
OverTime                    2000 non-null object
PerformanceRating           2000 non-null float64
RelationshipSatisfaction    2000 non-null float64
TotalWorkingYears           2000 non-null float64
YearsAtCompany              2000 non-null float64
dtypes: float64(10), object(5)
memory usage: 234.5+ KB


In [97]:
print('Breakdown of attrition. No = Stayed, Yes = Left')

df.Attrition.value_counts()

Breakdown of attrition. No = Stayed, Yes = Left


No     1680
Yes     320
Name: Attrition, dtype: int64

# Preprocessing

It's good that we don't have any null value. Let's encode the categorical data to ints. There are some categorical values those have been encoded once from the origin and transfered to type int. We want to use some of them for the cross features, so we want to transfer their type to string. 

In [98]:
to_convert = ['Education','EnvironmentSatisfaction','JobSatisfaction',
            'PerformanceRating','RelationshipSatisfaction']

for col in to_convert:
    df[col] = df[col].astype(np.str) 
   


Encode the categorical features:

In [99]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

to_encode = { 'Department','Gender','MaritalStatus','OverTime','Education','EnvironmentSatisfaction','JobSatisfaction',
            'PerformanceRating','RelationshipSatisfaction'}
encoders = dict()

for col in list(to_encode) +['Attrition']:
    if col=='Attrition':
        tmp = LabelEncoder()
        df[col] = tmp.fit_transform(df[col])
        df[col+'_int'] = tmp.fit_transform(df[col])
    else:
        encoders[col] = LabelEncoder()
        df[col+'_int'] = encoders[col].fit_transform(df[col])
    

Then, let's scale the numeric features. 

In [102]:
categorical_features =list(to_encode)
categorical_features = [x+'_int' for x in categorical_features]
numerics = set(df.columns) - to_encode
numerics = list(numerics - set(categorical_features)-{'Attrition'})
ss = StandardScaler()
for atr in numerics:
    df[atr] = df[atr].astype(np.float)    
    df[atr] = ss.fit_transform(df[atr].values.reshape(-1, 1))
    
feature_columns = categorical_features + numerics

In [103]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 25 columns):
Age                             2000 non-null float64
Attrition                       2000 non-null int64
Department                      2000 non-null object
DistanceFromHome                2000 non-null float64
Education                       2000 non-null object
EnvironmentSatisfaction         2000 non-null object
Gender                          2000 non-null object
JobSatisfaction                 2000 non-null object
MaritalStatus                   2000 non-null object
MonthlyIncome                   2000 non-null float64
OverTime                        2000 non-null object
PerformanceRating               2000 non-null object
RelationshipSatisfaction        2000 non-null object
TotalWorkingYears               2000 non-null float64
YearsAtCompany                  2000 non-null float64
MaritalStatus_int               2000 non-null int64
EnvironmentSatisfaction_int     2000 non-n

### Evaluation Metric

Let's take a moment to break down what the three main metrics mean in our task, i.e. predicting whether or not an employee will leave a company, and what that mean for our businesses using the model:

* accuracy: Values getting true positives and negatives.  This probably isn't very useful for our dataset because there are significantly fewer people that left the company than stayed

* precision: Values a low false positive rate.  This probably isn't the best either, because while we wouldn't **want** to think that an employee is leaving when they aren't, it probably won't hurt the business, unless the employer grossly overreacts and scares them away

* recall: Values a low false negative rate.  This is the best metric for our case.  If our job is to see when employees leave, and if the fact is that they usually **don't** leave, and if its potentially pretty damaging to the firm when the employee **does** leave, we want to make sure that we miss as few cases as possible.


### Validation Method

Because of the imbalance in our prediction label we'll use a stratified split, this way we'll preserve the distribution in our model.  In an attempt to realistically generalize the overall performance of our model we'll use a nested cross-validation scheme.  We'll use k-fold as opposed to a different cv scheme like shuffle-split, because our dataset is not that large and we would like to train on as much data as possible.  Using a k-fold cv ensures that we train on all of our data.  The inner loop will tune the hyper-parameters of our model which will be discussed later.

## Model Building

Let's just see how we well we can do with a singl network on all of our data.  We'll just split all of our data up into one train and test set.

In [128]:
from sklearn.model_selection import train_test_split

# stratified 90/10 train/test split`
df_train, df_test = train_test_split(df, test_size=0.1, stratify=df.Attrition)

X_train =  ss.fit_transform(df_train[feature_columns].values).astype(np.float32)
X_test =  ss.transform(df_test[feature_columns].values).astype(np.float32)

y_train = df_train['Attrition'].values.astype(np.int)
y_test = df_test['Attrition'].values.astype(np.int)

print('train', X_train.shape, 'test', X_test.shape)

train (1800, 15) test (200, 15)


In [21]:
#import some keras stuff
from keras.models import Sequential
from keras.layers import Dense, Activation, Input
from keras.layers import Embedding, Flatten, Merge, concatenate
from keras.models import Model

Using TensorFlow backend.


In [106]:
# This returns a tensor
inputs = Input(shape=(X_train.shape[1],))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(units=10, activation='relu')(inputs)
predictions = Dense(1,activation='sigmoid')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)

In [107]:
model.compile(optimizer='sgd',
              loss='mean_squared_error',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 15)                0         
_________________________________________________________________
dense_167 (Dense)            (None, 10)                160       
_________________________________________________________________
dense_168 (Dense)            (None, 1)                 11        
Total params: 171
Trainable params: 171
Non-trainable params: 0
_________________________________________________________________


In [108]:
%%time

model.fit(X_train, y_train, epochs=10, batch_size=50, verbose=0)

from sklearn import metrics as mt
yhat = np.round(model.predict(X_test))
print(mt.confusion_matrix(y_test,yhat),mt.recall_score(y_test,yhat))

[[158  10]
 [ 17  15]] 0.46875
Wall time: 2.55 s


Well that didn't do very well, but there are a few issues

1. We have a pretty big class imbalance and not a lot of positives, which makes it difficult to evaluate the loss function

2. We were using a mean squared error for our loss function on a binary classification task.  While this isn't terrible, it would likely work better if we used cross-entropy instead.

Luckily its easy to modify both of these in Keras.  The loss function is simple to change, and the fit function includes a parameter for 'class_weights' which accepts a dictionary of weights for each class value to use when computing the loss function.  This way we can tell the model which class is "more important".  Let's try it.

In [109]:
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])


model.fit(X_train, y_train, epochs=10, batch_size=50, verbose=0, class_weight={0 : 0.20, 1 : 0.80})

yhat = np.round(model.predict(X_test))
print(mt.confusion_matrix(y_test,yhat),mt.recall_score(y_test,yhat))

[[154  14]
 [  4  28]] 0.875


Much better, looks like we might want to tune that parameter of class weights

### Parameter tuning in Deep Network

We identify the following as parameters that we can tune

- Number of layers
- Number of neurons per layer
- The weights of the class in the loss function
- number of epochs

Keras has a wrapper class for a model to be used as an sklearn estimator which we can that pass to GridSearchCV.  The only caveat is that we must use Keras' Sequential Model to do so which means we can only have one input branch.  This is okay though because if we only build it on the deep side we can use the result of that with our wide, cross-category branch later on.

In [111]:
# taken from https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
# this uses the Keras Wrapper to make the model usable by sk-learn

# Use scikit-learn to grid search the batch size and epochs
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

# Function to create model, required for KerasClassifier
def create_model(num_neurons=12, input_dim=8):
    # create model
    model = Sequential()
    
    # num_neurons is a list of the number nuerons at each layer
    for layer, num in enumerate(num_neurons):
        if layer == 0:
            model.add(Dense(num, input_dim=input_dim, activation='relu'))
        else:
            model.add(Dense(units=num, activation='relu'))
    
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Now that we can creat models in that can be used by SKLearn, lets GridSearch over the parameters we identified.  Note that the number of neurons and number of layers is combined into one parameter called `num_neurons` which is a list of the number of output nerouns at each layer.

In [113]:
# this will move inside nested CV
from sklearn.model_selection import GridSearchCV
num_neurons = [[5, 10], [5, 10, 20]]
epochs = [10]
class_weight = [{0:x, 1:1-x} for x in np.linspace(0.1, 0.5, 2)]
param_grid = dict(num_neurons=num_neurons,
                  epochs=epochs,
                  class_weight=class_weight)

model = KerasClassifier(build_fn=create_model, input_dim=X_train.shape[-1], epochs=10, verbose=0)
g = GridSearchCV(estimator=model, param_grid=param_grid, verbose=3, scoring='recall')
r = g.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, epochs=10, num_neurons=[5, 10] 
[CV]  class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, epochs=10, num_neurons=[5, 10], score=1.0, total=   2.6s
[CV] class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, epochs=10, num_neurons=[5, 10] 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s


[CV]  class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, epochs=10, num_neurons=[5, 10], score=1.0, total=   2.7s
[CV] class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, epochs=10, num_neurons=[5, 10] 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.5s remaining:    0.0s


[CV]  class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, epochs=10, num_neurons=[5, 10], score=0.979381443298969, total=   3.5s
[CV] class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, epochs=10, num_neurons=[5, 10, 20] 
[CV]  class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, epochs=10, num_neurons=[5, 10, 20], score=0.9893617021276596, total=   3.7s
[CV] class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, epochs=10, num_neurons=[5, 10, 20] 
[CV]  class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, epochs=10, num_neurons=[5, 10, 20], score=1.0, total=   3.0s
[CV] class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, epochs=10, num_neurons=[5, 10, 20] 
[CV]  class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, epochs=10, num_neurons=[5, 10, 20], score=0.9381443298969072, total=   2.8s
[CV] class_weight={0: 0.5, 1: 0.5}, epochs=10, num_neurons=[5, 10] ...
[CV]  class_weight={0: 0.5, 1: 0.5}, epochs=10, num_neuro

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   41.1s finished


In [114]:
pd.DataFrame(r.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_class_weight,param_epochs,param_num_neurons,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,2.257783,0.788941,0.993127,0.968640,"{0: 0.1, 1: 0.9}",10,"[5, 10]","{'class_weight': {0: 0.1, 1: 0.9}, 'epochs': 1...",1,1.000000,0.989691,1.000000,0.958115,0.979381,0.958115,0.216405,0.160885,0.009720,0.014885
1,2.491110,0.774675,0.975835,0.986065,"{0: 0.1, 1: 0.9}",10,"[5, 10, 20]","{'class_weight': {0: 0.1, 1: 0.9}, 'epochs': 1...",2,0.989362,0.994845,1.000000,1.000000,0.938144,0.963351,0.319765,0.084749,0.027003,0.016199
2,2.568314,0.920839,0.295533,0.307155,"{0: 0.5, 1: 0.5}",10,"[5, 10]","{'class_weight': {0: 0.5, 1: 0.5}, 'epochs': 1...",4,0.000000,0.000000,0.886598,0.921466,0.000000,0.000000,0.163672,0.123668,0.417946,0.434383
3,2.917350,0.859220,0.894275,0.895000,"{0: 0.5, 1: 0.5}",10,"[5, 10, 20]","{'class_weight': {0: 0.5, 1: 0.5}, 'epochs': 1...",3,0.744681,0.721649,0.948454,0.963351,0.989691,1.000000,0.886745,0.070186,0.107110,0.123487


That did a lot better with the more divergent class weights and the greater number of neurons.

## Wide and Deep

Now let's combine a sparse branch with cross categories and a deep branch where we perform a grid search on the number of layers.

Let's pick some categories to cross that might contain some interesting information about the data.

In [115]:
cross_columns = [['Gender','MaritalStatus'],
                    ['Education', 'JobSatisfaction'],['Department','PerformanceRating'],
                    ['Education', 'JobSatisfaction','RelationshipSatisfaction'],['Department','OverTime'],
                ]

X_train_num =  df_train[numerics].values
X_test_num = df_test[numerics].values

Now we'll train the embeddings, and then take that result and run it through our GridSearched Deep Network

In [117]:
# we need to create separate sequential models for each embedding
embed_branches = []
X_ints_train = []
X_ints_test = []
all_inputs = []
all_branch_outputs = []


# reset this input branch
all_branch_outputs = []
# add in the embeddings
for col in categorical_features:
    # encode as ints for the embedding
    X_ints_train.append( df_train[col].values )
    X_ints_test.append( df_test[col].values )
    
    # get the number of categories
    N = max(X_ints_train[-1]+1) # same as the max(df_train[col])
    
    # create embedding branch from the number of categories
    inputs = Input(shape=(1,),dtype='int32', name=col)
    all_inputs.append(inputs)
    x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
    x = Flatten()(x)
    all_branch_outputs.append(x)
    
# also get a dense branch of the numeric features
all_inputs.append(Input(shape=(X_train_num.shape[1],),sparse=False,name='numeric_data'))
x = Dense(units=20, activation='relu')(all_inputs[-1])
all_branch_outputs.append( x )

# merge the branches together
deep_branch = concatenate(all_branch_outputs)
    
final_branch = Dense(units=1,activation='sigmoid')(deep_branch)

model = Model(inputs=all_inputs, outputs=final_branch)

model.compile(optimizer='adagrad',
              loss='mean_squared_error',
              metrics=['accuracy'])

model.fit(X_ints_train+ [X_train_num],
        y_train, epochs=10, batch_size=32, verbose=0)

In [118]:
# Grab the results right before the output layer, this is the realization of all the embedded data

model2 = Model(inputs=model.inputs, outputs=model.get_layer(index=-2).output)

X_embed = model2.predict(X_ints_train+ [X_train_num])

Now we can put that through the gridsearch

In [120]:
from sklearn.model_selection import GridSearchCV
num_neurons = [[5, 10], [5, 10, 20]]
class_weight=class_weight = [{0:x, 1:1-x} for x in np.linspace(0.1, 0.5, 2)]
param_grid = dict(num_neurons=num_neurons,
                  class_weight=class_weight)

model = KerasClassifier(build_fn=create_model, input_dim=X_embed.shape[-1], epochs=10, verbose=0)
g = GridSearchCV(estimator=model, param_grid=param_grid, verbose=3, scoring='recall')
r = g.fit(X_embed, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10] 
[CV]  class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10], score=1.0, total=   3.2s
[CV] class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10] 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.2s remaining:    0.0s


[CV]  class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10], score=1.0, total=   4.5s
[CV] class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10] 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    7.9s remaining:    0.0s


[CV]  class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10], score=1.0, total=   3.4s
[CV] class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10, 20] 
[CV]  class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10, 20], score=1.0, total=   3.3s
[CV] class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10, 20] 
[CV]  class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10, 20], score=1.0, total=   3.9s
[CV] class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10, 20] 
[CV]  class_weight={0: 0.10000000000000001, 1: 0.90000000000000002}, num_neurons=[5, 10, 20], score=1.0, total=   4.4s
[CV] class_weight={0: 0.5, 1: 0.5}, num_neurons=[5, 10] ..............
[CV]  class_weight={0: 0.5, 1: 0.5}, num_neurons=[5, 10], score=0.9680851063829787, total=   4.7s
[CV] class_weight={0: 0.5, 1: 0.5}, num_neurons=[5, 10] ..............
[CV]  cla

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:   52.8s finished


Let's look at how the best model did.

In [121]:
best_deep = r.best_estimator_.model
print(best_deep.summary())
best_deep_params = r.best_estimator_.get_params()
print(best_deep_params)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_258 (Dense)            (None, 5)                 700       
_________________________________________________________________
dense_259 (Dense)            (None, 10)                60        
_________________________________________________________________
dense_260 (Dense)            (None, 1)                 11        
Total params: 771
Trainable params: 771
Non-trainable params: 0
_________________________________________________________________
None
{'input_dim': 139, 'epochs': 10, 'verbose': 0, 'class_weight': {0: 0.10000000000000001, 1: 0.90000000000000002}, 'num_neurons': [5, 10], 'build_fn': <function create_model at 0x0000014E08D30730>}


Now we can use this model in our combined wide and deep network.

In [123]:
# we need to create separate sequential models for each embedding
embed_branches = []
X_ints_train = []
X_ints_test = []
all_inputs = []
all_branch_outputs = []

for cols in cross_columns:
    # encode crossed columns as ints for the embedding
    enc = LabelEncoder()
    
    # create crossed labels
    # needs to be commented better, Eric!
    X_crossed_train = df_train[cols].apply(lambda x: '_'.join(x), axis=1)
    X_crossed_test = df_test[cols].apply(lambda x: '_'.join(x), axis=1)
    
    enc.fit(np.hstack((X_crossed_train.values,  X_crossed_test.values)))
    X_crossed_train = enc.transform(X_crossed_train)
    X_crossed_test = enc.transform(X_crossed_test)
    X_ints_train.append( X_crossed_train )
    X_ints_test.append( X_crossed_test )
    
    # get the number of categories
    N = max(X_ints_train[-1]+1) # same as the max(df_train[col])
    
    # create embedding branch from the number of categories
    inputs = Input(shape=(1,),dtype='int32', name = '_'.join(cols))
    all_inputs.append(inputs)
    x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
    x = Flatten()(x)
    all_branch_outputs.append(x)
    
# merge the branches together
wide_branch = concatenate(all_branch_outputs)

# reset this input branch
all_branch_outputs = []
# add in the embeddings
for col in categorical_features:
    # encode as ints for the embedding
    X_ints_train.append( df_train[col].values )
    X_ints_test.append( df_test[col].values )
    
    # get the number of categories
    N = max(X_ints_train[-1]+1) # same as the max(df_train[col])
    
    # create embedding branch from the number of categories
    inputs = Input(shape=(1,),dtype='int32', name=col)
    all_inputs.append(inputs)
    x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
    x = Flatten()(x)
    all_branch_outputs.append(x)
    
# also get a dense branch of the numeric features
all_inputs.append(Input(shape=(X_train_num.shape[1],),sparse=False,name='numeric_data'))
x = Dense(units=20, activation='relu')(all_inputs[-1])
all_branch_outputs.append( x )

# merge the branches together
deep_branch = concatenate(all_branch_outputs)

# here is where we'll use the result of the GridSearch
for layer in best_deep.layers[:-1]:
    deep_branch = layer(deep_branch)
    
final_branch = concatenate([wide_branch, deep_branch])
final_branch = Dense(units=1,activation='sigmoid')(final_branch)

model = Model(inputs=all_inputs, outputs=final_branch)

model.compile(optimizer='adagrad',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_ints_train+ [X_train_num],
        y_train, epochs=10, batch_size=32, verbose=1, class_weight=best_deep_params['class_weight'])

Epoch 1/10
1800/1800 [==============================] - 1s - loss: 0.0913 - acc: 0.7072     
Epoch 2/10
1800/1800 [==============================] - 0s - loss: 0.0298 - acc: 0.9822     
Epoch 3/10
1800/1800 [==============================] - 0s - loss: 0.0118 - acc: 1.0000     
Epoch 4/10
1800/1800 [==============================] - 0s - loss: 0.0060 - acc: 1.0000     
Epoch 5/10
1800/1800 [==============================] - 0s - loss: 0.0036 - acc: 1.0000     
Epoch 6/10
1800/1800 [==============================] - 0s - loss: 0.0025 - acc: 1.0000     
Epoch 7/10
1800/1800 [==============================] - 0s - loss: 0.0018 - acc: 1.0000     
Epoch 8/10
1800/1800 [==============================] - 0s - loss: 0.0014 - acc: 1.0000     
Epoch 9/10
1800/1800 [==============================] - 0s - loss: 0.0011 - acc: 1.0000     
Epoch 10/10
1800/1800 [==============================] - 0s - loss: 9.3714e-04 - acc: 1.0000     


And finally, check out how it did.

In [124]:
yhat = np.round(model.predict(X_ints_test + [X_test_num]))
print(mt.confusion_matrix(y_test,yhat),mt.recall_score(y_test,yhat))

[[168   0]
 [  0  32]] 1.0


Well it got all of them right....  Let's see if we can generalize that using a nested grid search.

## Generalization using nested CV

In [134]:
from sklearn.model_selection import StratifiedKFold

outer_loop = StratifiedKFold(n_splits=5)

X = df.drop('Attrition', axis=1).values
y = df.Attrition.values

scores = []
i = 1

for train_idx, test_idx in outer_loop.split(X, y):
    
    # split data
    df_train = pd.DataFrame(X[train_idx], columns=df.columns.drop('Attrition'))
    df_test = pd.DataFrame(X[test_idx], columns=df.columns.drop('Attrition'))
    
    X_train = ss.fit_transform(df_train[feature_columns].values).astype(np.float32)
    X_test = ss.fit_transform(df_test[feature_columns].values).astype(np.float32)

    y_train = y[train_idx].astype(np.int)
    y_test = y[test_idx].astype(np.int)

    # the whole thing
    X_train_num =  df_train[numerics].values
    X_test_num = df_test[numerics].values

    
    # train deep
    print('\n************************************')    
    print('training embedings')
    print('************************************\n')

    # we need to create separate sequential models for each embedding
    embed_branches = []
    X_ints_train = []
    X_ints_test = []
    all_inputs = []
    all_branch_outputs = []


    # reset this input branch
    all_branch_outputs = []
    # add in the embeddings
    for col in categorical_features:
        # encode as ints for the embedding
        X_ints_train.append( df_train[col].values )
        X_ints_test.append( df_test[col].values )

        # get the number of categories
        N = max(X_ints_train[-1]+1) # same as the max(df_train[col])

        # create embedding branch from the number of categories
        inputs = Input(shape=(1,),dtype='int32', name=col)
        all_inputs.append(inputs)
        x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
        x = Flatten()(x)
        all_branch_outputs.append(x)

    # also get a dense branch of the numeric features
    all_inputs.append(Input(shape=(X_train_num.shape[1],),sparse=False,name='numeric_data'))
    x = Dense(units=20, activation='relu')(all_inputs[-1])
    all_branch_outputs.append( x )

    # merge the branches together
    deep_branch = concatenate(all_branch_outputs)

    final_branch = Dense(units=1,activation='sigmoid')(deep_branch)

    model = Model(inputs=all_inputs, outputs=final_branch)

    model.compile(optimizer='adagrad',
                  loss='mean_squared_error',
                  metrics=['accuracy'])

    model.fit(X_ints_train+ [X_train_num],
            y_train, epochs=10, batch_size=32, verbose=1)
    
    
    # get output from embedding
    model2 = Model(inputs=model.inputs, outputs=model.get_layer(index=-2).output)

    X_embed = model2.predict(X_ints_train+ [X_train_num])
    
    # Grid Search Deep
    print('\n************************************')
    print('Grid Searching the layers of the Deep')
    print('************************************\n')

    from sklearn.model_selection import GridSearchCV
    num_neurons = [[5, 10], [5, 10, 20], [15, 15, 10]]
    class_weight=class_weight = [{0:x, 1:1-x} for x in np.linspace(0.1, 0.5, 3)]
    param_grid = dict(num_neurons=num_neurons,
                      class_weight=class_weight)

    model = KerasClassifier(build_fn=create_model, input_dim=X_embed.shape[-1], epochs=10, verbose=0)
    g = GridSearchCV(estimator=model, param_grid=param_grid, verbose=1, scoring='recall')
    r = g.fit(X_embed, y_train)
    
    best_deep = r.best_estimator_.model
    best_deep_params = r.best_estimator_.get_params()
    
    
    # Wide and best deep
    
    # we need to create separate sequential models for each embedding
    embed_branches = []
    X_ints_train = []
    X_ints_test = []
    all_inputs = []
    all_branch_outputs = []

    for cols in cross_columns:
        # encode crossed columns as ints for the embedding
        enc = LabelEncoder()

        # create crossed labels
        # needs to be commented better, Eric!
        X_crossed_train = df_train[cols].apply(lambda x: '_'.join(x), axis=1)
        X_crossed_test = df_test[cols].apply(lambda x: '_'.join(x), axis=1)

        enc.fit(np.hstack((X_crossed_train.values,  X_crossed_test.values)))
        X_crossed_train = enc.transform(X_crossed_train)
        X_crossed_test = enc.transform(X_crossed_test)
        X_ints_train.append( X_crossed_train )
        X_ints_test.append( X_crossed_test )

        # get the number of categories
        N = max(X_ints_train[-1]+1) # same as the max(df_train[col])

        # create embedding branch from the number of categories
        inputs = Input(shape=(1,),dtype='int32', name = '_'.join(cols))
        all_inputs.append(inputs)
        x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
        x = Flatten()(x)
        all_branch_outputs.append(x)

    # merge the branches together
    wide_branch = concatenate(all_branch_outputs)

    # reset this input branch
    all_branch_outputs = []
    # add in the embeddings
    for col in categorical_features:
        # encode as ints for the embedding
        X_ints_train.append( df_train[col].values )
        X_ints_test.append( df_test[col].values )

        # get the number of categories
        N = max(X_ints_train[-1]+1) # same as the max(df_train[col])

        # create embedding branch from the number of categories
        inputs = Input(shape=(1,),dtype='int32', name=col)
        all_inputs.append(inputs)
        x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
        x = Flatten()(x)
        all_branch_outputs.append(x)

    # also get a dense branch of the numeric features
    all_inputs.append(Input(shape=(X_train_num.shape[1],),sparse=False,name='numeric_data'))
    x = Dense(units=20, activation='relu')(all_inputs[-1])
    all_branch_outputs.append( x )

    # merge the branches together
    deep_branch = concatenate(all_branch_outputs)

    # here is where we'll use the result of the GridSearch
    for layer in best_deep.layers[:-1]:
        deep_branch = layer(deep_branch)

    final_branch = concatenate([wide_branch, deep_branch])
    final_branch = Dense(units=1,activation='sigmoid')(final_branch)

    model = Model(inputs=all_inputs, outputs=final_branch)

    model.compile(optimizer='adagrad',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    print('\n************************************')
    print('fitting the whole thing')
    print('*************************************\n')

    model.fit(X_ints_train+ [X_train_num],
            y_train, epochs=10, batch_size=32, verbose=1, class_weight=best_deep_params['class_weight'])
    
    yhat = np.round(model.predict(X_ints_test + [X_test_num]))
    score = mt.recall_score(y_test,yhat)
    print('\n*************************************')
    print('Overall Accuracy loop', i, ":", score)
    print('*************************************\n')
    scores.append(score)
    i = i + 1

C:\Users\Cameron\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)



************************************
training embedings
************************************

Epoch 1/10
1600/1600 [==============================] - 2s - loss: 0.1222 - acc: 0.8962     
Epoch 2/10
1600/1600 [==============================] - 0s - loss: 0.0591 - acc: 0.9756     
Epoch 3/10
1600/1600 [==============================] - 0s - loss: 0.0367 - acc: 0.9912     
Epoch 4/10
1600/1600 [==============================] - 0s - loss: 0.0251 - acc: 0.9925     
Epoch 5/10
1600/1600 [==============================] - 0s - loss: 0.0184 - acc: 0.9938     
Epoch 6/10
1600/1600 [==============================] - 0s - loss: 0.0140 - acc: 0.9963     
Epoch 7/10
1600/1600 [==============================] - 0s - loss: 0.0111 - acc: 0.9975     
Epoch 8/10
1600/1600 [==============================] - 0s - loss: 0.0090 - acc: 0.9981     
Epoch 9/10
1600/1600 [==============================] - 0s - loss: 0.0075 - acc: 0.9981     
Epoch 10/10
1600/1600 [==============================] - 0s - loss: 

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  2.5min finished



************************************
fitting the whole thing
*************************************

Epoch 1/10
1600/1600 [==============================] - 3s - loss: 0.0796 - acc: 0.7600     
Epoch 2/10
1600/1600 [==============================] - 0s - loss: 0.0165 - acc: 0.9963     
Epoch 3/10
1600/1600 [==============================] - 0s - loss: 0.0063 - acc: 0.9994     
Epoch 4/10
1600/1600 [==============================] - 0s - loss: 0.0034 - acc: 0.9994     
Epoch 5/10
1600/1600 [==============================] - 0s - loss: 0.0022 - acc: 0.9994     
Epoch 6/10
1600/1600 [==============================] - 0s - loss: 0.0015 - acc: 0.9994     
Epoch 7/10
1600/1600 [==============================] - 0s - loss: 0.0011 - acc: 1.0000     
Epoch 8/10
1600/1600 [==============================] - 0s - loss: 8.8282e-04 - acc: 1.0000     
Epoch 9/10
1600/1600 [==============================] - 0s - loss: 7.1550e-04 - acc: 1.0000     
Epoch 10/10
1600/1600 [==============================]

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  3.0min finished


...

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  4.3min finished



************************************
fitting the whole thing
*************************************

Epoch 1/10
1600/1600 [==============================] - 5s - loss: 0.0976 - acc: 0.6038     
Epoch 2/10
1600/1600 [==============================] - 0s - loss: 0.0191 - acc: 0.9919     
Epoch 3/10
1600/1600 [==============================] - 0s - loss: 0.0051 - acc: 0.9981     
Epoch 4/10
1600/1600 [==============================] - 0s - loss: 0.0025 - acc: 1.0000     
Epoch 5/10
1600/1600 [==============================] - 0s - loss: 0.0015 - acc: 1.0000     
Epoch 6/10
1600/1600 [==============================] - 0s - loss: 0.0011 - acc: 1.0000     
Epoch 7/10
1600/1600 [==============================] - 0s - loss: 8.0446e-04 - acc: 1.0000     
Epoch 8/10
1600/1600 [==============================] - 0s - loss: 6.3273e-04 - acc: 1.0000     
Epoch 9/10
1600/1600 [==============================] - 0s - loss: 5.1717e-04 - acc: 1.0000     
Epoch 10/10
1600/1600 [===========================

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  5.0min finished



************************************
fitting the whole thing
*************************************

Epoch 1/10
1600/1600 [==============================] - 6s - loss: 0.1040 - acc: 0.8800     
Epoch 2/10
1600/1600 [==============================] - 0s - loss: 0.0302 - acc: 0.9994     
Epoch 3/10
1600/1600 [==============================] - 0s - loss: 0.0119 - acc: 1.0000     
Epoch 4/10
1600/1600 [==============================] - 0s - loss: 0.0063 - acc: 1.0000     - ETA: 0s - loss: 0.0076 - a
Epoch 5/10
1600/1600 [==============================] - ETA: 0s - loss: 0.0041 - acc: 1.000 - 0s - loss: 0.0040 - acc: 1.0000     
Epoch 6/10
1600/1600 [==============================] - 0s - loss: 0.0029 - acc: 1.0000     
Epoch 7/10
1600/1600 [==============================] - 0s - loss: 0.0022 - acc: 1.0000     
Epoch 8/10
1600/1600 [==============================] - 0s - loss: 0.0017 - acc: 1.0000     
Epoch 9/10
1600/1600 [==============================] - 0s - loss: 0.0014 - acc: 1.0000  

InvalidArgumentError: indices[15,0] = 532 is not in [0, 531)
	 [[Node: embedding_241/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](embedding_241/embeddings/read, _arg_RelationshipSatisfaction_int_21_0_13)]]

Caused by op 'embedding_241/Gather', defined at:
  File "C:\Users\Cameron\Anaconda3\envs\mlenv\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Cameron\Anaconda3\envs\mlenv\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Cameron\Anaconda3\envs\mlenv\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\Cameron\Anaconda3\envs\mlenv\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Cameron\Anaconda3\envs\mlenv\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File 

I don't what that error is^ but it finished everything.

In [135]:
print('Mean performance:', np.mean(scores), "(+-", np.std(scores), ")")

Mean performance: 1.0 (+- 0.0 )


It looked like it did pretty well.  Really so obviously this isn't a very difficult probelm, even though we expanded the dataset with some random data, although this probably did help to regularize the data.

When compared to the plain MLP that we ran at the beginning of the notebook